In [54]:
pwd

'/Users/cohlem/Projects/Experimentation/RL-GRPO'

In [16]:
from torch.testing._internal.common_distributed import spawn_threads_and_init_comms

import os
import torch
import os
from torch import distributed as dist
import torch.nn as nn
from torch.distributed.device_mesh import init_device_mesh
# from torch.distributed.fsdp import FullyShardedDataParallel as FSDP, ShardingStrategy
from torch.distributed._composable.fsdp import fully_shard
from torch.distributed.tensor.parallel import parallelize_module, ColwiseParallel, RowwiseParallel, SequenceParallel, PrepareModuleInput
from torch.distributed._tensor import Shard, Replicate, distribute_tensor, DTensor

# from llama2_model import Transformer, ModelArgs

import torch.distributed as dist
import torch
from torch.distributed._tensor import DeviceMesh, Shard, Replicate, distribute_tensor


from torch.distributed._composable.fsdp import fully_shard

# Let's fucking goooo

### Data loader

 - load data from huggingface hub using hf datasets
 - make a dataloader (shuffle data, set batch sizes and so on)
 

In [4]:
from datasets import load_dataset
train_dataset = load_dataset('CohleM/rlpr_test_small', split='train')

In [5]:
from torch.utils.data import DataLoader

In [6]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, path):
        self.data = load_dataset(path, split='train')
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        question = self.data['prompt'][idx]
        answer = self.data['reward_model'][idx]
        return {'question' : question , 'answer' : answer }

In [9]:
train_dataset = CustomDataset('CohleM/rlpr_test_small')

In [10]:
len(train_dataset)

50

### training algorithm



```python

mesh = init_device_mesh(3,2) # 3 DDP, 2 TP



train_dataloader = DataLoader(the_dataset, shuffle=True, batch_size=64)
for train_data in train_dataloader: # each train_data is of len batch_size=64
    # split the data into different 3 DDP groups

    complete_rollout = []
    step = 0
    
    
    for each_data in train_data:
        each_data = each_data.reapeat(rollout.n)
        
        # generate responses.
        responses = llm.generate(each_data)
        advantages = calculate_advantages(each_data, responses)
        complete_rollout.append(each_data, responses)
    
    old_llm = make_copy_of_current_llm()    
    for update in range(no_of_updates_per_ppo_step):
        minibatches = DataLoader(complete_rollout, batch_size=ppo_mini_batch_size) # no shuffle here
        for mini_batch in minibatches:
            # generate the logprobs 
            old_logprobs = old_llm(mini_batchl)
            logprobs = llm(mini_batch)
            ppo_loss = calculate_ppo_loss(old_logprobs, logprobs, advantages) # i.e ratio * adv, where ratio = logprobs/ old_logprobs
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
            
        
        
    # we're doing GRPO so for each question generate,  
    
    
```

### training algorithm



```python

mesh = init_device_mesh(3,2) # 3 DDP, 2 TP



train_dataloader = DataLoader(the_dataset, shuffle=True, batch_size=64)
for train_data in train_dataloader: # each train_data is of len batch_size=64
    # split the data into different 3 DDP groups

    
    step = 0
    
    
    # repeat the train_data X rollout.n = new_train_data
    # split the new_train_data into different 3 DDP groups processes, each will get (len(new_train_data) / DDP_size) data
    new_train_data = train_data.repeat(rollout.n)
        
    #split here, i.e new_train_data into 3 ddp groups,
    # we also need an estimate of how many data each group can process at a time, lets say it's vllm_mini_batch
    new_train_data = DataLoader(new_train_data, vllm_mini_batch)
    complete_rollout = []
    for vllm_batch in new_train_data:
        # generate responses.
        responses = llm.generate(vllm_batch)
        complete_rollout.append(vllm_batch,responses)
    
     
    
    # all_gather data into one process then caculate the return/advantages,
    # after gathering complete_rollout should be full.
    
    advantages = calculate_advantages(each_data, responses)
    complete_rollout.append(each_data, responses)
    
    
    old_llm = make_copy_of_current_llm()  # we only need model parameters for old_llm, and will have the same sharding strategy as the current llm i.e llm.
    
    for update in range(no_of_updates_per_ppo_step):
        minibatches = DataLoader(complete_rollout, batch_size=ppo_mini_batch_size) # no shuffle here, or maybe split the mini_batches 
        for mini_batch in minibatches:
            # generate the logprobs 
            old_logprobs = old_llm(mini_batch)
            logprobs = llm(mini_batch)
            ppo_loss = calculate_ppo_loss(old_logprobs, logprobs, advantages) # i.e ratio * adv, where ratio = logprobs/ old_logprobs
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
            
     # update the vllm model with our llm's weight.
    
        
    # we're doing GRPO so for each question generate,  
    
    
```

In [151]:
torch.arange(50).unsqueeze(0).shape

torch.Size([1, 50])

# PPO trainer

dummy

In [27]:
from dataclasses import dataclass

@dataclass
class Config:
    train_batch_size: int = 64
    rollout_n: int = 8
    mini_batch_size: int = 8
    model_name: str = 'Qwen/Qwen2.5-0.5B-Instruct'
    ddp_size: int = 2
    tp_size: int = 2
    lr: float = 1e-6
#     train_batch_size: int = 64
        
config = Config()


# from transformers import AutoTokenizer, AutoModel
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed._composable.fsdp import fully_shard
from torch.distributed.tensor.parallel import parallelize_module, ColwiseParallel, RowwiseParallel, SequenceParallel, PrepareModuleInput
from llama2_model import Transformer, ModelArgs

class Worker:
    """
    This is the policy that we will be updating with each gradient update, we rollout using this policy's
    parameters, and we use the logprobs from this policy, we will also copy it's weights to make it old policy
    """
    
    def __init__(self, config):
        
#         self.tokenizer = AutoTokenizer.from_pretrained(config.model_name)
        self.config = config
        device= 'cuda' if torch.cuda.is_available() else 'cpu'
        
        # define model from huggingface later on
        simple_llama2_config = ModelArgs(dim=4, n_layers=1, n_heads=4, vocab_size=8)
        self.model = Transformer.from_model_args(simple_llama2_config).to(device)

        # first make a device mesh
        fsdp_size = int(int(os.environ['WORLD_SIZE']) / (config.ddp_size * config.tp_size))
        self.mesh = init_device_mesh(device,(config.ddp_size,fsdp_size, config.tp_size), mesh_dim_names=["DDP", "FSDP", "TP"])

#         # shard model's parameters on tp axis
#         self.prepare_tp_model()

#         # shard model's parameter on dp axis
#         prepare_dp_model()

    def prepare_optimizer(self):
        if config.tp_size > 1:
            self.model = prepare_tp_model(self.model, self.mesh)
        
        self.model = prepare_dp_model(self.model, self.mesh)
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.config.lr)
#         if dist.get_rank() == 0:
#             print(f' after dp model rank: {dist.get_rank()} attention wq {self.model.layers[0].attention.wq.weight}')
        
        
def prepare_tp_model(model, mesh):

    layer_tp_plan = {
        # Now the input and output of SequenceParallel has Shard(1) layouts,
        # to represent the input/output tensors sharded on the sequence dimension
        ### ------ The module names will be different for new models -----, take a look at model.named_parameters()
        "attention_norm": SequenceParallel(),
        "attention": PrepareModuleInput(
            input_layouts=(Shard(1), Replicate()),
            desired_input_layouts=(Replicate(), Replicate()),
        ),
        "attention.wq": ColwiseParallel(use_local_output=False),
        "attention.wk": ColwiseParallel(use_local_output=False),
        "attention.wv": ColwiseParallel(use_local_output=False),
        "attention.wo": RowwiseParallel(output_layouts=Shard(1)),
        "ffn_norm": SequenceParallel(),
        "feed_forward": PrepareModuleInput(
            input_layouts=(Shard(1),),
            desired_input_layouts=(Replicate(),),
        ),
        "feed_forward.w1": ColwiseParallel(),
        "feed_forward.w2": RowwiseParallel(output_layouts=Shard(1)),
        "feed_forward.w3": ColwiseParallel(),
    }


    # Apply TP
    for layer_id, transformer_block in enumerate(model.layers):

        parallelize_module(
            module=transformer_block,
            device_mesh=mesh['TP'],
            parallelize_plan=layer_tp_plan,
        )

    model = parallelize_module(
        model,
        mesh['TP'],
        {
            "tok_embeddings": RowwiseParallel(
                input_layouts=Replicate(),
                output_layouts=Shard(1),
            ),
            "norm": SequenceParallel(),
            "output": ColwiseParallel(
                input_layouts=Shard(1),
                output_layouts=Replicate()
            ),
        }
    )
    
    return model
    
def prepare_dp_model(model, mesh):
    for layer in model.layers:
        fully_shard(layer, mesh=mesh['DDP','FSDP'])

    sharded_model = fully_shard(model, mesh=mesh['DDP','FSDP'])
    return sharded_model


class Actor(Worker):
    def __init__(self, config):
        super().__init__(config)
        device= 'cuda' if torch.cuda.is_available() else 'cpu'
        
#         define model from huggingface later on
        simple_llama2_config = ModelArgs(dim=4, n_layers=1, n_heads=4, vocab_size=8)
        self.model = Transformer.from_model_args(simple_llama2_config).to(device)
        
        # actor will need optimizer
        self.prepare_optimizer()


class Rollout(Worker):
    def __init__(self, config):
        super().__init__(config)
            
        # init model using sglang
        self.prepare_env_var()

        if self.mesh["TP"].get_local_rank() == 0:
            os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
            engine = Engine(
                    model_path=self.config.model_name,
                    dtype="bfloat16",
                    tp_size=self.mesh["TP"].size(),
                    mem_fraction_static=0.5,
                    # enable_memory_saver=True,
                    port=30000 + dist.get_rank()
                )
                
                
    def prepare_env_var(self):
        if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys(): # remove the use of common store for communication
            del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
        monkey_patch_torch_reductions()
        
        
        # THE reason for doing this is because, we'll store rollout worker's (sglang) weight in these TP group
        # otherwise, SGL will use all the available cuda devices.
        cuda_visible_devices = [None] * self.config.tp_size
        dist.all_gather_object(cuda_visible_devices, os.environ['LOCAL_RANK'], self.mesh['TP'].get_group())
        os.environ['CUDA_VISIBLE_DEVICES'] = ",".join(cuda_visible_devices)

        
  
class Trainer:
    def __init__(self, config):
        self.config = config
        self.actor = Actor(config)
#         self.rollout = Rollout(config)
        
        
    def train():
        # construct train dataloader
        train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=self.config.train_batch_size)
        for train_batch in train_dataloader:
            # generate rollouts.
            # for now use the model.generate() from huggingface
            
        
import os
from torch import optim
from torch.distributed.checkpoint.state_dict import (
    StateDictOptions, get_model_state_dict, get_state_dict
)

@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    os.environ['WORLD_SIZE'] = str(world_size)
    os.environ['LOCAL_RANK'] = str(dist.get_rank())
    
    # can't use os.environ['LOCAL_RANK'] in spawn_threads_and_init_comms so using dist.get_rank() which gives the
    # local rank, but instead we must to os.environ['LOCAL_RANK'] otherwise.
    
    local_rank = dist.get_rank()
#     local_rank = int(os.environ["LOCAL_RANK"])
    
    # if gpu set this
    # torch.cuda.set_device(local_rank)
    
    ppo_trainer = Trainer(config)
    


In [28]:
world_size = 4
shard_big_tensor(world_size)

In [57]:
class MyDataset(Dataset):
    def __init__(self):
        
        self.data = [
            {"tokens": [1, 4]},
            {"tokens": [1, 3]},
            {"tokens": [1,5]}
        ]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]


dataset = MyDataset()
loader = DataLoader(dataset, batch_size=2, collate_fn=collate_fn)

for batch in loader:
    print(batch)


[{'tokens': [1, 4]}, {'tokens': [1, 4]}, {'tokens': [1, 4]}, {'tokens': [1, 4]}, {'tokens': [1, 4]}, {'tokens': [1, 4]}, {'tokens': [1, 4]}, {'tokens': [1, 4]}, {'tokens': [1, 3]}, {'tokens': [1, 3]}, {'tokens': [1, 3]}, {'tokens': [1, 3]}, {'tokens': [1, 3]}, {'tokens': [1, 3]}, {'tokens': [1, 3]}, {'tokens': [1, 3]}]
[{'tokens': [1, 5]}, {'tokens': [1, 5]}, {'tokens': [1, 5]}, {'tokens': [1, 5]}, {'tokens': [1, 5]}, {'tokens': [1, 5]}, {'tokens': [1, 5]}, {'tokens': [1, 5]}]


In [53]:
import copy

In [56]:
def collate_fn(batch):
    return [copy.deepcopy(ex) for ex in batch for _ in range(8)]


In [106]:
import copy
# from RL2.datasets.base import BaseDataset, load_dataset
from torch.utils.data import Dataset
class RLDataset(Dataset):
    
    def __init__(self, data_path, responses_per_prompt):

        self.dataset = load_dataset(data_path, split='train')
        self.responses_per_prompt = responses_per_prompt

    def __getitem__(self, idx):

        ex = self.dataset[idx]
        messages = ex["messages"]
        answer = ex["answer"]

        return {
            "messages": messages,
            "answer": answer
        }
    def __len__(self):
        return len(self.dataset)



def collate_fn(batch):
    responses_per_prompt = 4
    return [
        copy.deepcopy(ex)
        for ex in batch
        for _ in range(responses_per_prompt)
    ]

In [107]:
test_dataset = RLDataset('CohleM/olympiad_small', 4)

In [109]:
from torchdata.stateful_dataloader import StatefulDataLoader

In [110]:
data_loader = StatefulDataLoader(test_dataset, batch_size = 3, drop_last=True, collate_fn=collate_fn)

### now for each entires in the data, apply chat template, generate output, generate response_mask, generate reward.

In [144]:
def rollout(tokenizer, data_list):
    
    for data in data_list:
        messages,answer = data['messages'], data['answer']
        ans = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        print(ans)
        
        # now generate it through sglang, one benefit of sglang is that we don't don't 
        
    

In [145]:
for data in data_loader:
    print(len(data))
#     print(data)
    rollout(tokenizer, data)
    break
    

12
<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Xenia and Sergey play the following game. Xenia thinks of a positive integer $N$ not exceeding 5000. Then she fixes 20 distinct positive integers $a_{1}, a_{2}, \ldots, a_{20}$ such that, for each $k=1,2, \ldots, 20$, the numbers $N$ and $a_{k}$ are congruent modulo $k$. By a move, Sergey tells Xenia a set $S$ of positive integers not exceeding 20 , and she tells him back the set $\left\{a_{k}: k \in S\right\}$ without spelling out which number corresponds to which index. How many moves does Sergey need to determine for sure the number Xenia thought of?<|im_end|>
<|im_start|>assistant

<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
Xenia and Sergey play the following game. Xenia thinks of a positive integer $N$ not exceeding 5000. Then she fixes 20 distinct positive integers $a_{1}, a_{2}, \ldots, a_{20

In [147]:
chunks = [[1, 2], [3, 4], [5]]
flattened = sum(chunks, [])
print(flattened)  # [1, 2, 3, 4, 5]

[1, 2, 3, 4, 5]


In [113]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(config.model_name)


In [116]:
ans = tokenizer.encode('this is cool', add_special_tokens=False)

In [119]:
question = tokenizer.encode('This is question', add_special_tokens=False)

In [120]:
states = []

states.extend(question)
states.extend(ans)

In [122]:
new_states = torch.LongTensor(states[:-1])

In [124]:
states

[1986, 374, 3405, 574, 374, 7010]

In [127]:
(new_states.shape[-1] - 1) * [0] + [1]

[0, 0, 0, 0, 1]

actor changes the model


# rollout and update works


In [ ]:

import os
import torch
from torch import distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from sglang.srt.entrypoints.engine import Engine
import sglang as sgl
import nest_asyncio
from sglang.test.test_utils import DEFAULT_SMALL_MODEL_NAME_FOR_TEST
from transformers import AutoModel
from sglang.srt.patch_torch import monkey_patch_torch_reductions
from transformers import AutoModelForCausalLM

from torch.distributed._composable.fsdp import fully_shard
from torch.distributed._tensor import (DTensor, Replicate, Shard,
                                       distribute_tensor)
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed.tensor.parallel import (ColwiseParallel,
                                               PrepareModuleInput,
                                               RowwiseParallel,
                                               SequenceParallel,
                                               parallelize_module)


from torch.distributed.checkpoint.state_dict import (
    StateDictOptions, get_model_state_dict, get_state_dict
)

from sglang.srt.utils import MultiprocessingSerializer
from sglang.srt.model_executor.model_runner import LocalSerializedTensor


def setup():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    backend = 'nccl' if device == 'cuda' else 'gloo'
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    local_rank = int(os.environ["LOCAL_RANK"])
    if device == 'cuda':
        torch.cuda.set_device(local_rank)
    os.environ["NCCL_CUMEM_ENABLE"] = "0"
    os.environ["NCCL_NVLS_ENABLE"] = "0"
    # Initialize with explicit parameters
    dist.init_process_group(
        backend=backend, 
        world_size=world_size,
        rank=rank
    )

def prepare_llama_tp_layer(layer, device_mesh):

    parallelize_plan = {
        "input_layernorm": SequenceParallel(),
        "self_attn.q_proj": ColwiseParallel(),
        "self_attn.k_proj": ColwiseParallel(),
        "self_attn.v_proj": ColwiseParallel(),
        "self_attn.o_proj": RowwiseParallel(
            output_layouts=Shard(1)
        ),
        "post_attention_layernorm": SequenceParallel(),
        "mlp.gate_proj": ColwiseParallel(),
        "mlp.up_proj": ColwiseParallel(),
        "mlp.down_proj": RowwiseParallel(
            output_layouts=Shard(1)
        )
    }
    parallelize_module(
        module=layer,
        device_mesh=device_mesh,
        parallelize_plan=parallelize_plan
    )

def prepare_environment_variables(mesh):
    if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys():
        del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
    monkey_patch_torch_reductions()
    cuda_visible_devices = mesh["TP"].size() * [None]
    dist.all_gather_object(
        cuda_visible_devices,
        os.environ["LOCAL_RANK"],
        mesh["TP"].get_group()
    )
    # print(f' GLOBAL RNAK {dist.get_rank()} devices {cuda_visible_devices} ')
    os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(cuda_visible_devices)

def start():
    # nest_asyncio.apply()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    local_rank = dist.get_rank()
    torch.cuda.set_device(local_rank)
    # torch.cuda.synchronize()
    mesh = init_device_mesh(device, (1,2,2), mesh_dim_names = ['DDP','FSDP', 'TP'])

    # print(mesh)
    # -- rollout ---
    prepare_environment_variables(mesh)
    # -- rollout ---
    
    # print(f"Global rank {dist.get_rank()}, local rank {os.environ['LOCAL_RANK']} mesh {mesh['TP']} visible devices {os.environ["CUDA_VISIBLE_DEVICES"]}\n\n  ")
    # print(torch.cuda.device_count())

    model_name = "ibm-granite/granite-3.3-2b-base"

    # ------- ROLLOUT ---------
    if mesh["TP"].get_local_rank() == 0:
        os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
        engine = Engine(
                model_path=model_name,
                dtype="bfloat16",
                tp_size=mesh["TP"].size(),
                mem_fraction_static=0.5,
                # enable_memory_saver=True,
                port=30000 + dist.get_rank()
            )

        # print(engine)
        param_name = 'model.layers.0.self_attn.q_proj.weight'
        # if dist.get_rank() == 0:
        #     breakpoint()

        # reserved_gb = torch.cuda.memory_reserved() / (1024 ** 3)
        # total_gb = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        # param = engine.get_weights_by_name(param_name)[0][:5]
        # print(f"Global rank {dist.get_rank()}  param : {param} \n\n  ")
    # ------- ROLLOUT ---------

    print(f'gloabal rank {dist.get_rank()} memory allocated {torch.cuda.memory_allocated() }')
        
    # dist.barrier()

    

    # ----- rollout -----
    # Do the rollout
    if mesh["TP"].get_local_rank() == 0:
        # release memory temporarily.
        engine.release_memory_occupation()
        torch.cuda.empty_cache()
    # ----- rollout -----

    print(f'after release memory oocupation rank {dist.get_rank()} memory allocated {torch.cuda.memory_allocated() }')
    
    # load the actor model
    model = AutoModelForCausalLM.from_pretrained(model_name)
    # print(model)
    print(f'After automodel rank {dist.get_rank()} memory allocated {torch.cuda.memory_allocated() }')
    
    for layer in model.model.layers:
        prepare_llama_tp_layer(layer, mesh['TP'])

    
# ----- outer block --------
    parallelize_plan = {
        "model.embed_tokens": ColwiseParallel(
            output_layouts=Shard(1)
        ),
        "model.norm": SequenceParallel(),
        "lm_head": ColwiseParallel()
    }
    parallelize_module(
        module=model,
        device_mesh=mesh['TP'],
        parallelize_plan=parallelize_plan
    )
# ----- outer block --------

# ----------FSDP -----------
    for layer in model.model.layers:
        fully_shard(layer, mesh=mesh['DDP', 'FSDP'])
    
    fully_shard(model, mesh=mesh['DDP', 'FSDP'])

    # print(f' GLOBAL RANK {dist.get_rank()} {model.model.layers[0].self_attn.q_proj.weight.to_local().shape} memory allocated {torch.cuda.memory_allocated() / (1024 **3) }')



    # -- rollout --
    # resume memory occupation
    torch.cuda.empty_cache()
    if mesh["TP"].get_local_rank() == 0:
        engine.resume_memory_occupation()


    # -- actor changes the model---
    # let's make some changes to the first and last row
    # -- actor changes the model---
    # let's make some changes to the first and last row
    new_var = torch.tensor([1.5] * 5).to(torch.cuda.current_device())
    # print(model.model.layers[0].self_attn.q_proj.weight)
    # print(model.model.layers[0].self_attn.q_proj.weight[0][:5

    old_w = model.model.layers[0].self_attn.q_proj.weight
    local_weight = old_w.to_local().detach().clone()  
    local_weight[0][:5] = new_var

    new_mesh = old_w.device_mesh            # reuse original mesh
    placements = old_w.placements         # usually (Shard(0),)

    new_dt = DTensor.from_local(
        local_weight,
        device_mesh=new_mesh,
        placements=placements,
        run_check=False                    # safe because we kept the shape
    )

    model.model.layers[0].self_attn.q_proj.weight = torch.nn.Parameter(new_dt)

    print('after replacement', model.model.layers[0].self_attn.q_proj.weight)


    # offload to cpu
    options = StateDictOptions(full_state_dict=False, cpu_offload=True)
    state_dict = get_model_state_dict(
        model, options=options
    )
    
    for idx, (name, tensor) in enumerate(state_dict.items()):
        # load to gpu again
        tensor = tensor.to(torch.cuda.current_device())
        # print(name)
        # if name == 'model.layers.0.self_attn.q_proj.weight':
            
        serialized_tensor = MultiprocessingSerializer.serialize(tensor.full_tensor() if isinstance(tensor, DTensor) else tensor)
        serialized_tensors = [None] * mesh['TP'].size() if mesh['TP'].get_local_rank() == 0 else None
        
        dist.gather_object(serialized_tensor, serialized_tensors, group_dst=0, group=mesh['TP'].get_group())
        
        if mesh["TP"].get_local_rank() == 0:
            # print(serialized_tensors)
            engine.update_weights_from_tensor(named_tensors=[(name, LocalSerializedTensor(values=serialized_tensors))])

        
        # print(f"rank {dist.get_rank()} seriliazed_tensor {serialized_tensor.shape} len_ST: {len(serialized_tensors) if isinstance(serialized_tensors,list) else serialized_tensors} ")

    dist.barrier()

    if mesh["TP"].get_local_rank() == 0:
        param_start = engine.get_weights_by_name(param_name)[0][:5]
        # param_end = engine.get_weights_by_name(param_name)[-1][:5]
        print(f"Global rank {dist.get_rank()}  param after weight update : {param_start} \n\n  ")
            
            

    return

        
def main():
    setup()
    start()
    dist.destroy_process_group()


if __name__ == '__main__':
    main()

# NEW TEST


In [1]:
import os
import torch
from dataclasses import dataclass
from torch import distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from datasets import load_dataset
import math
import copy
from torch.utils.data import Dataset
from torchdata.stateful_dataloader import StatefulDataLoader
from torch.testing._internal.common_distributed import spawn_threads_and_init_comms

In [14]:



class RLDataset(Dataset):

    def __init__(self, data_path, responses_per_prompt):

        self.dataset = load_dataset(data_path, split='train')
        self.responses_per_prompt = responses_per_prompt

    def __getitem__(self, idx):

        ex = self.dataset[idx]
        messages = ex["messages"]
        answer = ex["answer"]

        return {
            "messages": messages,
            "answer": answer
        }
    def __len__(self):
        return len(self.dataset)



    def collate_fn(self, batch):

        return [
            copy.deepcopy(ex)
            for ex in batch
            for _ in range(self.responses_per_prompt)
        ]



def setup():
    # device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = 'cpu' 
    backend = 'nccl' if device == 'cuda' else 'gloo'
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    local_rank = int(os.environ["LOCAL_RANK"])
    if device == 'cuda':
        torch.cuda.set_device(local_rank)
    os.environ["NCCL_CUMEM_ENABLE"] = "0"
    os.environ["NCCL_NVLS_ENABLE"] = "0"
    # Initialize with explicit parameters
    dist.init_process_group(
        backend=backend, 
        world_size=world_size,
        rank=rank
    )


def split_data_list(data_list, mesh):
  # we need to scatter this data_list across this mesh group, from local group 0.

    rank = mesh.get_local_rank()
    size = mesh.size()

    if rank == 0:
        data_per_ddp = math.ceil(len(data_list)/size)
    
    lists = [data_list[i * data_per_ddp: (i+1)* data_per_ddp] if rank ==0 else None for i in range(size)]
    
    lst = [None] # this is the output list
    dist.scatter_object_list(lst, lists, src=None, group_src=0, group=mesh.get_group())

    # print(f'rank {dist.get_rank()} got this list{lst}' )
    return lst[0]


def gather_data_list(data_list, mesh):
    # we need to scatter this data_list across this mesh group, from local group 0.

    rank = mesh.get_local_rank()
    size = mesh.size()

    lists = [None for i in range(size)] if rank==0 else None # Must be None on non-dst ranks otherwise it will call dist.gather_object in other ranks as well is None, it will be called only in the group_dst rank


    dist.gather_object(data_list,lists, group_dst=0, group=mesh.get_group()) 

    #   print(f'rank {dist.get_rank()} got this list{lists}' )
    return sum(lists, []) if rank==0 else None # if not group destination, lists wil be None, won't sum



def check_mem_allocated(rank, msg):
    ans = torch.cuda.memory_allocated() / (1024**3)
    print(f'RANK {rank} MEMORY_ALLOCATED {msg} {ans}')


def start():
    
    device = "cpu" 
    mesh = init_device_mesh(device, (2,2,2), mesh_dim_names = ['DDP','FSDP', 'TP'])

    train_data = RLDataset("CohleM/olympiad_small", 4)
    train_dataloader = StatefulDataLoader(train_data, batch_size=3, drop_last=True, collate_fn=train_data.collate_fn)
    # construct train dataloader
    
    for train_batch in train_dataloader:
        if mesh['TP'].get_local_rank() == 0:
            # then split across ddp
            data_list = split_data_list(train_batch, mesh=mesh['DDP'])
        
            print(f'rank {dist.get_rank()} data len {len(data_list)} first element {data_list[0]} \n')
            break

    return

@dataclass
class Config:
    train_batch_size: int = 64
    mini_batch_size: int = 8
    model_name: str = 'Qwen/Qwen2.5-0.5B-Instruct'
    ddp_size: int = 2 
    tp_size: int = 1 
    lr: float = 1e-6
    data_path: str = 'CohleM/olympiad_small'
    responses_per_prompt: int = 4
    per_rollout_size: int = 3
#     train_batch_size: int = 64
#  
# def main():
#     # setup process groups.
#     setup()

#     # Initialize ppo trainer with some config
#     start()
#     dist.destroy_process_group()


# if __name__ == '__main__':
#     main()

In [51]:
@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    
    device = "cpu" 
    model_device_mesh = init_device_mesh(device, (1,2,2), mesh_dim_names = ['DDP','FSDP','TP'])
    
    device_mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['DP', 'TP'])


    train_data = RLDataset("CohleM/olympiad_small", 4)
    print(device_mesh)
    train_dataloader = StatefulDataLoader(train_data, batch_size=3, drop_last=True, collate_fn=train_data.collate_fn)
    # construct train dataloader
    
#     print(f'rnak {dist.get_rank()} {mesh['dp']}')
    data_list = None
    for train_batch in train_dataloader:
        if device_mesh['TP'].get_local_rank() == 0:
            # then split across ddp
            data_list = split_data_list(train_batch, mesh=device_mesh['DP'])
        

            
        
        if device_mesh['TP'].get_local_rank() == 0:
            data_list = gather_data_list(data_list, mesh=device_mesh['DP'])
            
#             print(f'rank {dist.get_rank()} data len {len(data_list) if isinstance(data_list, list) else None} \n')
#         print(f'rank {dist.get_rank()} data len {len(data_list) if isinstance(data_list, list) else None} \n')
        
        # now for the actor 
        if device_mesh['TP'].get_local_rank() == 0:
            # then split across ddp
            data_list = split_data_list(train_batch, mesh=device_mesh['DP'])
            
        data_list = broadcast_data_list(data_list, mesh=device_mesh['TP'])
        
        print(f'rank {dist.get_rank()} data len {len(data_list) if isinstance(data_list, list) else None} first element {data_list[0]}\n')

        
        break
    return

In [52]:

def broadcast_data_list(data_list, mesh):

    
    if mesh.get_local_rank() == 0:
        len_data_list = torch.tensor(len(data_list))
    else:
        len_data_list = torch.tensor(0)
    
    dist.broadcast(len_data_list, group=mesh.get_group(), group_src=0)
    
#     print(len_data_list.item())


    if mesh.get_local_rank() != 0:
        data_list = [None for _ in range(len_data_list)]
    
    dist.broadcast_object_list(data_list, group=mesh.get_group(), group_src=0)

    return data_list



In [53]:
shard_big_tensor(4)

DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('DP', 'TP'))
DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('DP', 'TP'))
DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('DP', 'TP'))
DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('DP', 'TP'))
rank 0 data len 6 first element {'messages': [{'content': 'Xenia and Sergey play the following game. Xenia thinks of a positive integer $N$ not exceeding 5000. Then she fixes 20 distinct positive integers $a_{1}, a_{2}, \\ldots, a_{20}$ such that, for each $k=1,2, \\ldots, 20$, the numbers $N$ and $a_{k}$ are congruent modulo $k$. By a move, Sergey tells Xenia a set $S$ of positive integers not exceeding 20 , and she tells him back the set $\\left\\{a_{k}: k \\in S\\right\\}$ without spelling out which number corresponds to which index. How many moves does Sergey need to determine for sure the number Xenia thought of?', 'role': 'user'}], 'answer': '2'}

rank 2 data len 6 first element {'messages': [{'content': 'Given a positive in

# Single layer model

In [161]:
import torch.nn as nn

In [224]:
import math

def split_data_list(data_list, mesh):
  # we need to scatter this data_list across this mesh group, from local group 0.

  rank = mesh.get_local_rank()
  size = mesh.size()

  if rank == 0:
    data_per_ddp = math.ceil(len(data_list)/size)
  
  lists = [data_list[i * data_per_ddp: (i+1)* data_per_ddp] if rank ==0 else None for i in range(size)]
  
  lst = [None] # this is the output list
  dist.scatter_object_list(lst, lists, src=None, group_src=0, group=mesh.get_group())

  # print(f'rank {dist.get_rank()} got this list{lst}' )
  return lst[0]


def gather_data_list(data_list, mesh):
  # we need to scatter this data_list across this mesh group, from local group 0.

  rank = mesh.get_local_rank()
  size = mesh.size()

  lists = [None for i in range(size)] if rank==0 else None # Must be None on non-dst ranks otherwise it will call dist.gather_object in other ranks as well is None, it will be called only in the group_dst rank
  
  
  dist.gather_object(data_list,lists, group_dst=0, group=mesh.get_group()) 

  print(f'rank {dist.get_rank()} got this list{lists}' )
  return lists



In [225]:
import math

def split_data_list(data_list, mesh):
  # we need to scatter this data_list across this mesh group, from local group 0.

  rank = mesh.get_local_rank()
  size = mesh.size()

  if rank == 0:
    data_per_ddp = math.ceil(len(data_list)/size)
  
  lists = [data_list[i * data_per_ddp: (i+1)* data_per_ddp] if rank ==0 else None for i in range(size)]
  
  lst = [None] # this is the output list
  dist.scatter_object_list(lst, lists, src=None, group_src=0, group=mesh.get_group())

  # print(f'rank {dist.get_rank()} got this list{lst}' )
  return lst[0]


def gather_data_list(data_list, mesh):
  # we need to scatter this data_list across this mesh group, from local group 0.

  rank = mesh.get_local_rank()
  size = mesh.size()

  lists = [None for i in range(size)] if rank==0 else None # Must be None on non-dst ranks otherwise it will call dist.gather_object in other ranks as well is None, it will be called only in the group_dst rank
  
  
  dist.gather_object(data_list,lists, group_dst=0, group=mesh.get_group()) 

  print(f'rank {dist.get_rank()} got this list{lists}' )
  return lists




data_list = [torch.ones((2,2)), torch.ones((2,2))*2]


### ERROR

In [464]:

@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    
    device = "cpu" 
    model_device_mesh = init_device_mesh(device, (1,2,2), mesh_dim_names = ['DDP','FSDP','TP'])
    
    device_mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['dp', 'tp'])

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.layer0 = nn.Linear(4,4,bias=False)
        def forward(self,x):
            return self.layer0(x)
        
    model = Model()

    model.layer0.weight = nn.Parameter(torch.arange(1.,17.).reshape(4,4))
    
    parallelize_module(
        module=model,
        device_mesh=model_device_mesh['TP'],
        parallelize_plan={
            "layer0": RowwiseParallel() # since rowwise parallel will replicate the output layouts, each tp rank will get the same outptu.
        }
    )

    data_list = [torch.ones((2,2)), torch.ones((2,2))*2]
    x = split_data_list(data_list, device_mesh['dp'])[0]


    fully_shard(model.layer0, mesh=model_device_mesh['DDP','FSDP'])
    fully_shard(model, mesh=model_device_mesh['DDP','FSDP'])
    
    optim = torch.optim.AdamW(model.parameters(), lr=1e-6)


    optim.zero_grad(set_to_none=True)
    out = model(x)
    print(out)
    dist.barrier()
    torch.manual_seed(42)
    original = torch.randn_like(out)
    loss = (original - out)**2
    loss = loss.sum()
    
#     print(f'rank {dist.get_rank()} loss {loss} ')
    
    print(f' RANK {dist.get_rank()} x = {x} weights = {model.layer0.weight} \n\n')
    loss.backward()

    
#     print(f' RANK {dist.get_rank()} x = {x} \n\n')
#     print(f' RANK {dist.get_rank()} x = {x} weights = {model.layer0.weight.grad} \n\n')
    
    return

In [465]:
shard_big_tensor(4)

E0815 17:23:01.822000 76359 site-packages/torch/testing/_internal/common_distributed.py:1275] Caught exception: 
E0815 17:23:01.822000 76359 site-packages/torch/testing/_internal/common_distributed.py:1275] Traceback (most recent call last):
E0815 17:23:01.822000 76359 site-packages/torch/testing/_internal/common_distributed.py:1275]   File "/Users/cohlem/anaconda3/envs/deep_learning/lib/python3.12/site-packages/torch/testing/_internal/common_distributed.py", line 1065, in worker
E0815 17:23:01.822000 76359 site-packages/torch/testing/_internal/common_distributed.py:1275]     callback()
E0815 17:23:01.822000 76359 site-packages/torch/testing/_internal/common_distributed.py:1275]   File "/Users/cohlem/anaconda3/envs/deep_learning/lib/python3.12/site-packages/torch/testing/_internal/common_distributed.py", line 1088, in <lambda>
E0815 17:23:01.822000 76359 site-packages/torch/testing/_internal/common_distributed.py:1275]     threads = _run_test_method_with_multi_threads(world_size, lambd

AsyncCollectiveTensor(tensor([[10., 26., 42., 58.],
        [10., 26., 42., 58.]]))
AsyncCollectiveTensor(tensor([[ 20.,  52.,  84., 116.],
        [ 20.,  52.,  84., 116.]]))
AsyncCollectiveTensor(tensor([[10., 26., 42., 58.],
        [10., 26., 42., 58.]]))
AsyncCollectiveTensor(tensor([[ 20.,  52.,  84., 116.],
        [ 20.,  52.,  84., 116.]]))
 RANK 1 x = tensor([[1., 1.],
        [1., 1.]]) weights = DTensor(local_tensor=tensor([[3., 4.],
        [7., 8.]]), device_mesh=DeviceMesh('cpu', [[[0, 1], [2, 3]]], mesh_dim_names=('DDP', 'FSDP', 'TP')), placements=(Replicate(), Shard(dim=0), Shard(dim=1))) 


 RANK 3 x = tensor([[2., 2.],
        [2., 2.]]) weights = DTensor(local_tensor=tensor([[11., 12.],
        [15., 16.]]), device_mesh=DeviceMesh('cpu', [[[0, 1], [2, 3]]], mesh_dim_names=('DDP', 'FSDP', 'TP')), placements=(Replicate(), Shard(dim=0), Shard(dim=1))) 


 RANK 0 x = tensor([[1., 1.],
        [1., 1.]]) weights = DTensor(local_tensor=tensor([[1., 2.],
        [5., 6.]])

RuntimeError: Thread 1 exited with exception:
Traceback (most recent call last):
  File "/Users/cohlem/anaconda3/envs/deep_learning/lib/python3.12/site-packages/torch/testing/_internal/common_distributed.py", line 1065, in worker
    callback()
  File "/Users/cohlem/anaconda3/envs/deep_learning/lib/python3.12/site-packages/torch/testing/_internal/common_distributed.py", line 1088, in <lambda>
    threads = _run_test_method_with_multi_threads(world_size, lambda: func(self, *args, **kwargs))
                                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/pv/2c84c3jx48v8485vx92jq6zc0000gn/T/ipykernel_76359/4070643146.py", line 50, in shard_big_tensor
    loss.backward()
  File "/Users/cohlem/anaconda3/envs/deep_learning/lib/python3.12/site-packages/torch/_tensor.py", line 648, in backward
    torch.autograd.backward(
  File "/Users/cohlem/anaconda3/envs/deep_learning/lib/python3.12/site-packages/torch/autograd/__init__.py", line 353, 

In [279]:

@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    
    device = "cpu" 
    model_device_mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['FSDP','TP'])
    
    device_mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['dp', 'tp'])

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.layer0 = nn.Linear(4,4,bias=False)
        def forward(self,x):
            return self.layer0(x)
        
    model = Model()

    
    print(model)
    
    model.layer0.weight = nn.Parameter(torch.arange(1.,17.).reshape(4,4))
    
    parallelize_module(
        module=model,
        device_mesh=model_device_mesh['TP'],
        parallelize_plan={
            "layer0": RowwiseParallel() # since rowwise parallel will replicate the output layouts, each tp rank will get the same outptu.
        }
    )

    
    x = split_data_list(data_list, device_mesh['dp'])[0]
    

    fully_shard(model, mesh=model_device_mesh['FSDP'])
    
    optim = torch.optim.AdamW(model.parameters(), lr=1e-6)

    
    out = model(x)
    dist.barrier()

#     print(f'RANK {dist.get_rank()}', model_device_mesh['FSDP'])
#     print(f'RANK {dist.get_rank()}', device_mesh['dp'])

#     print(f' RANK {dist.get_rank()} x = {x} \n\n')
    loss = out.sum()
    optim.zero_grad(set_to_none=True)
    loss.backward()
    
    
    print(f' RANK {dist.get_rank()} x = {x} weights = {out.wait()} \n\n')
    
    return

In [280]:
shard_big_tensor(4)

Model(
  (layer0): Linear(in_features=4, out_features=4, bias=False)
)Model(
  (layer0): Linear(in_features=4, out_features=4, bias=False)
)
Model(
  (layer0): Linear(in_features=4, out_features=4, bias=False)
)
Model(
  (layer0): Linear(in_features=4, out_features=4, bias=False)
)

RANK 3 DeviceMesh('cpu', [1, 3], mesh_dim_names=('FSDP',))
RANK 0 DeviceMesh('cpu', [0, 2], mesh_dim_names=('FSDP',))
RANK 2 DeviceMesh('cpu', [0, 2], mesh_dim_names=('FSDP',))
RANK 1 DeviceMesh('cpu', [1, 3], mesh_dim_names=('FSDP',))
RANK 3 DeviceMesh('cpu', [1, 3], mesh_dim_names=('dp',))
RANK 1 DeviceMesh('cpu', [1, 3], mesh_dim_names=('dp',))
RANK 2 DeviceMesh('cpu', [0, 2], mesh_dim_names=('dp',))
RANK 0 DeviceMesh('cpu', [0, 2], mesh_dim_names=('dp',))
 RANK 0 x = tensor([[1., 1.],
        [1., 1.]]) 


 RANK 2 x = tensor([[2., 2.],
        [2., 2.]]) 


 RANK 1 x = tensor([[1., 1.],
        [1., 1.]]) 


 RANK 3 x = tensor([[2., 2.],
        [2., 2.]]) 


 RANK 1 x = tensor([[1., 1.],
        [1., 1

In [281]:
data_list

[tensor([[1., 1.],
         [1., 1.]]),
 tensor([[2., 2.],
         [2., 2.]])]

In [365]:
from torch.distributed import ReduceOp

In [412]:
data_list = [torch.ones((2,2)), torch.ones((2,2))*2]

@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    
    device = "cpu" 
    model_device_mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['FSDP','TP'])
    
    device_mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['dp', 'tp'])

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.layer0 = nn.Linear(4,4,bias=False)
        def forward(self,x):
            return self.layer0(x)
        
    model = Model()
    
    model.layer0.weight = nn.Parameter(torch.arange(1.,17.).reshape(4,4))
    
    parallelize_module(
        module=model,
        device_mesh=model_device_mesh['TP'],
        parallelize_plan={
            "layer0": RowwiseParallel() # since rowwise parallel will replicate the output layouts, each tp rank will get the same outptu.
        }
    )
    
    optim = torch.optim.AdamW(model.parameters(), lr=1e-6)

    optim.zero_grad(set_to_none=True)
    
    if model_device_mesh['FSDP'].get_local_rank() ==0: # give one data
        print(f'rank {dist.get_rank()} x = {data_list[0]}')
        out0 = model(data_list[0])
#         print(out0)
        dist.barrier()
        torch.manual_seed(42)
        original = torch.randn_like(out0)
        loss = (original - out0)**2
        loss = loss.sum()
        print(f'rank {dist.get_rank()} loss: {loss}')
        loss.backward()
    
    dist.barrier()
#     print(f' RANK {dist.get_rank()} before second weights = {(model.layer0.weight.grad)} \n\n')
    
    if model_device_mesh['FSDP'].get_local_rank() ==1: # give one data
        print(f'rank {dist.get_rank()} x = {data_list[1]}')
        
        out1 = model(data_list[1])
#         print(out1)
        dist.barrier()
        torch.manual_seed(42)
        original = torch.randn_like(out1)
        loss = (original - out1)**2
        loss = loss.sum()
        print(f'rank {dist.get_rank()} loss: {loss}')
        loss.backward()

    dist.barrier()
    
#     print(f' RANK {dist.get_rank()} after second weights = {model.layer0.weight.grad} \n\n')

    
    dt = model.layer0.weight.grad.to_local()

    

    dist.all_reduce(dt, op=ReduceOp.SUM, group=model_device_mesh['FSDP'].get_group())

    print(f' RANK {dist.get_rank()} reduced weights = {dt/2} \n\n')  

    return

In [413]:
shard_big_tensor(4)

rank 0 x = tensor([[1., 1.],
        [1., 1.]])rank 1 x = tensor([[1., 1.],
        [1., 1.]])

rank 1 loss: 11675.63671875
rank 0 loss: 11675.63671875
rank 3 x = tensor([[2., 2.],
        [2., 2.]])
rank 2 x = tensor([[2., 2.],
        [2., 2.]])
rank 3 loss: 46960.45703125
rank 2 loss: 46960.45703125
 RANK 1 reduced weights = tensor([[102.3585, 102.3585],
        [260.1725, 260.1725],
        [412.6720, 412.6720],
        [581.2230, 581.2230]]) 


 RANK 2 reduced weights = tensor([[102.3585, 102.3585],
        [260.1725, 260.1725],
        [412.6720, 412.6720],
        [581.2230, 581.2230]]) 


 RANK 3 reduced weights = tensor([[102.3585, 102.3585],
        [260.1725, 260.1725],
        [412.6720, 412.6720],
        [581.2230, 581.2230]]) 


 RANK 0 reduced weights = tensor([[102.3585, 102.3585],
        [260.1725, 260.1725],
        [412.6720, 412.6720],
        [581.2230, 581.2230]]) 




In [407]:
data_list

[tensor([[1., 1.],
         [1., 1.]]),
 tensor([[2., 2.],
         [2., 2.]])]

In [447]:
data_list = [torch.ones((2,2)), torch.ones((2,2))*2]

@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    
    device = "cpu" 
    model_device_mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['FSDP','TP'])
    
    device_mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['dp', 'tp'])

    class Model(nn.Module):
        def __init__(self):
            super().__init__()
            self.layer0 = nn.Linear(4,4,bias=False)
        def forward(self,x):
            return self.layer0(x)
        
    model = Model()

    
    print(model)
    
    model.layer0.weight = nn.Parameter(torch.arange(1.,17.).reshape(4,4))
    
    parallelize_module(
        module=model,
        device_mesh=model_device_mesh['TP'],
        parallelize_plan={
            "layer0": RowwiseParallel() # since rowwise parallel will replicate the output layouts, each tp rank will get the same outptu.
        }
    )

    
    x = split_data_list(data_list, device_mesh['dp'])[0]
    print(f' RANK {dist.get_rank()} x= {x} \n\n')
    

    fully_shard(model, mesh=model_device_mesh['FSDP'])
    
    optim = torch.optim.AdamW(model.parameters(), lr=1e-6)

    optim.zero_grad(set_to_none=True)
    out = model(x)
    print(out)
    dist.barrier()
    torch.manual_seed(42)
    original = torch.randn_like(out)
    loss = (original - out)**2
    loss = loss.sum()
    
    print(f'rank {dist.get_rank()} loss {loss} ')
    loss.backward()

    
    
    print(f' RANK {dist.get_rank()} weights = {model.layer0.weight.grad} \n\n')
    
    return

In [448]:
shard_big_tensor(4)

Model(
  (layer0): Linear(in_features=4, out_features=4, bias=False)
)
Model(
  (layer0): Linear(in_features=4, out_features=4, bias=False)
)
Model(
  (layer0): Linear(in_features=4, out_features=4, bias=False)
)
Model(
  (layer0): Linear(in_features=4, out_features=4, bias=False)
)
 RANK 2 x= tensor([[2., 2.],
        [2., 2.]]) 


 RANK 0 x= tensor([[1., 1.],
        [1., 1.]]) 


 RANK 3 x= tensor([[2., 2.],
        [2., 2.]]) 


 RANK 1 x= tensor([[1., 1.],
        [1., 1.]]) 


AsyncCollectiveTensor(tensor([[10., 26., 42., 58.],
        [10., 26., 42., 58.]]))
AsyncCollectiveTensor(tensor([[ 20.,  52.,  84., 116.],
        [ 20.,  52.,  84., 116.]]))
AsyncCollectiveTensor(tensor([[ 20.,  52.,  84., 116.],
        [ 20.,  52.,  84., 116.]]))
AsyncCollectiveTensor(tensor([[10., 26., 42., 58.],
        [10., 26., 42., 58.]]))
rank 1 loss 11675.63671875 
rank 2 loss 46960.45703125 
rank 0 loss 11675.63671875 
rank 3 loss 46960.45703125 
 RANK 0 weights = DTensor(local_tensor=tensor([[

In [463]:
data_list = [torch.ones((2,4)), torch.ones((2,4))*2]
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer0 = nn.Linear(4,4,bias=False)
    def forward(self,x):
        return self.layer0(x)

model = Model()


print(model)

model.layer0.weight = nn.Parameter(torch.arange(1.,17.).reshape(4,4))



optim = torch.optim.AdamW(model.parameters(), lr=1e-6)

optim.zero_grad(set_to_none=True)
out0 = model(data_list[0])
torch.manual_seed(42)
original = torch.randn_like(out0)
loss = (original - out0)**2
loss = loss.sum()

loss.backward()

out1 = model(data_list[1])
torch.manual_seed(42)
original = torch.randn_like(out1)
loss = (original - out1)**2
loss = loss.sum()

loss.backward()


    
print(f'weights = {model.layer0.weight.grad/2} \n\n')

Model(
  (layer0): Linear(in_features=4, out_features=4, bias=False)
)
weights = tensor([[102.3585, 102.3585, 102.3585, 102.3585],
        [260.1725, 260.1725, 260.1725, 260.1725],
        [412.6720, 412.6720, 412.6720, 412.6720],
        [581.2230, 581.2230, 581.2230, 581.2230]]) 




In [62]:
pos_emb = torch.tensor(torch.randn(2,4), torch.randn(2,5)])

ValueError: only one element tensors can be converted to Python scalars

In [67]:
torch.tensor([2,3])

tensor([2, 3])

In [68]:
[torch.randn(3,2)]

[tensor([[-1.0605,  0.4071],
         [ 1.3052,  0.4336],
         [-1.0205,  1.7036]])]

In [63]:
torch.tensor([torch.randn(2,4)])

ValueError: only one element tensors can be converted to Python scalars

In [55]:
import pickle

In [61]:
with open('/Users/cohlem/Downloads/pkls/minibatches_.pkl0', 'rb') as f:
    minibatch = pickle.load(f)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [71]:
import torch
ppo_loss = torch.randint(0,5, (2,3))

In [72]:
ppo_loss

tensor([[4, 3, 3],
        [1, 0, 4]])

In [73]:
ppo_loss.sum(dim=None)

tensor(15)